# FFTW C2C Cython Sequential

To avoid srun errors, it is necessary to create a stacked conda environment, containing some updated packages (.../env3 can have another name):

    conda create --prefix .../env3 python=3.9

Stack and install packages

    conda activate --stack .../env3
    conda install -c conda-forge cython pyfftw mpi4py mpi4py-fft

Activate when using the environment

    conda activate --stack .../env3

In [17]:
%%writefile cc2cs.pyx
#cython: boundscheck=False, wraparound=False, cdivision=True
#cython: initializedcheck=False, language_level=3, infer_types=True
def ff():
    import numpy as np, pyfftw as pf, time as tm
    
    t0 = tm.time()    # time measurement

    # data
    L = M = N = 576
    u = pf.empty_aligned( (N, M, L), dtype=np.complex128 )
    for k in range (u.shape[2]) :
        for j in range(u.shape[1]) :
            for i in range(u.shape[0]) :
                u[i, j, k] = i + j + k + 3
    u.real = np.sin ( u.real )
    u.imag = 0
    
    t1 = tm.time()    # time measurement

    # FFT
    uf = pf.interfaces.numpy_fft.fftn(u,
            overwrite_input=True, auto_contiguous=False,
            auto_align_input=False)
    # checksum
    s = np.sum(uf)
    
    t2 = tm.time()    # time measurement
    
    return s, L, t0, t1, t2

Overwriting cc2cs.pyx


In [18]:
%%writefile setups.py
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext
setup(
    name='cc2cs',
    ext_modules=[
        Extension('cc2cs',
            sources=['cc2cs.pyx'],
            extra_compile_args=['-O3']
        )
    ],
    cmdclass = {'build_ext': build_ext}
)

Overwriting setups.py


In [19]:
%%bash
rm -f cc2cs*.so
python setups.py build_ext --inplace

running build_ext
cythoning cc2cs.pyx to cc2cs.c
building 'cc2cs' extension
/scratch/ampemi/xxxx.xxxx/env2/bin/x86_64-conda_cos6-linux-gnu-cc -Wno-unused-result -Wsign-compare -DNDEBUG -O2 -Wall -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /scratch/ampemi/xxxx.xxxx/env3/include -fno-semantic-interposition -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /scratch/ampemi/xxxx.xxxx/env3/include -fno-semantic-interposition -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /scratch/ampemi/xxxx.xxxx/env2/include -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /scratch/ampemi/xxxx.xxxx/env2/include -fPIC -I/scratch/ampemi/xxxx.xxxx/env3/include/python3.9 -c cc2cs.c -o build/temp.linux-x86_64-3.9/cc2cs.o -O3
x86_64-conda_cos6-linux-gnu-gcc -pthread -shared -Wl,-O2 -Wl,--so


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [20]:
! ls cc2cs*.so

cc2cs.cpython-39-x86_64-linux-gnu.so


In [21]:
%%writefile cc2cs_c.py
import numpy as np
import time as tm
import cc2cs

t3 = tm.time()    # time measurement

s, L, t0, t1, t2 = cc2cs.ff()

t4 = tm.time()    # time measurement

print(f"S:{s*1E-5:.0f}", end='')
print(f" | L:{L:0g}", end='')
print(f" | T1:{t1-t0:.4f}", end='')
print(f" | TF:{t2-t1:.4f}", end='')
print(f" | TT:{t2-t0:.4f}", end='')
print(f" | TO:{t4-t3:.4f}")

Overwriting cc2cs_c.py


In [22]:
! time python cc2cs_c.py

S:270-0j | L:576 | T1:93.1251 | TF:22.6156 | TT:115.7406 | TO:117.0237


## Copy to /scratch

In [23]:
%%bash
rm /scratch${PWD#"/prj"}/cc2cs*
cp cc2cs* /scratch${PWD#"/prj"}

In [24]:
%%writefile cc2cs.srm
#!/bin/bash
#SBATCH --job-name cc2cs       # Job name
#SBATCH --partition cpu_small  # Select partition
#SBATCH --ntasks=1             # Total tasks
#SBATCH --time=00:05:00        # Limit execution time
#SBATCH --exclusive            # Exclusive acccess to nodes

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- # of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- # of tasks:' $SLURM_NTASKS
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
cd $SLURM_SUBMIT_DIR
echo -n '- List of nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST

# Environment
echo '-- modules ----------------------------'
cd
dir=/scratch${PWD#"/prj"}
cd $dir
source $dir/env2/etc/profile.d/conda.sh
conda activate $dir/env2
conda activate --stack $dir/env3
cd $dir/fft
                                              
# Executable
EXEC="python cc2cs_c.py"

# Start
echo '-- run --------------------------------'
echo '$ srun --mpi=pmi2 -n' $SLURM_NTASKS ${EXEC##*/}
echo '-- output -----------------------------'
srun --mpi=pmi2 -n $SLURM_NTASKS $EXEC
echo '~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

Overwriting cc2cs.srm


## Check

In [25]:
! sbatch --partition=cpu_dev cc2cs.srm

Submitted batch job 1333365


In [28]:
! squeue -n cc2cs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS
           1333365    cpu_dev   R  0:48     1   24


In [29]:
! squeue -n cc2cs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS


In [30]:
! cat /scratch${PWD#"/prj"}/slurm-1333365.out

- Job ID: 1333365
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1243
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 1 python cc2cs_c.py
-- output -----------------------------
S:270-0j | L:576 | T1:85.6123 | TF:19.5309 | TT:105.1432 | TO:106.0234
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


## Run

In [31]:
! sbatch cc2cs.srm
! sbatch cc2cs.srm
! sbatch cc2cs.srm

Submitted batch job 1333376
Submitted batch job 1333377
Submitted batch job 1333378


In [32]:
! squeue -n cc2cs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS
           1333376  cpu_small  PD  0:00     1    1
           1333377  cpu_small  PD  0:00     1    1
           1333378  cpu_small  PD  0:00     1    1


In [1]:
! squeue -n cc2cs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS


In [2]:
! cat /scratch${PWD#"/prj"}/slurm-1333376.out
! cat /scratch${PWD#"/prj"}/slurm-1333377.out
! cat /scratch${PWD#"/prj"}/slurm-1333378.out

- Job ID: 1333376
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1275
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 1 python cc2cs_c.py
-- output -----------------------------
S:270-0j | L:576 | T1:92.2451 | TF:18.9971 | TT:111.2422 | TO:115.0666
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1333377
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1275
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 1 python cc2cs_c.py
-- output -----------------------------
S:270-0j | L:576 | T1:87.9824 | TF:18.9791 | TT:106.9615 | TO:107.0620
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1333378
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: 

## Version

In [3]:
! cython --version

Cython version 0.29.21


In [7]:
! python --version

Python 3.9.4
